Módulos

In [ ]:
# Ferramentas do Selenium para WebScraping
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Manipulação de Dados
import pandas as pd 

# Recurso necessário para a função lastnum_sdot()
import re

# Limpar terminal
import os

Funções

In [ ]:
# Função para limpar o terminal
def limpar():
     os.system('cls')
     
# Status de quanto falta para completar a coleta
def carregamento(contador):
    max = 645
    count = contador
    print(f"Está faltando {max - count} para completar {max} ({count}/{max})")

# Pegar o último número exato com ponto dentro de uma string 
def lastnum_sdot(text):
        texto = text
        padrao = r'\b\d+(\.\d+)?\b$'
        match = re.search(padrao, texto)
        if match:
            numero = match.group(0)
        else:
            print("Nenhum número encontrado.")
        numero_sem_ponto = numero.replace('.', '')
        return int(numero_sem_ponto)

Abrir navegador no site da SSP para pegar os dados

In [ ]:
# Abrir o Google
navegador = webdriver.Chrome()

# Entrar no Site
navegador.get("https://www.ssp.sp.gov.br/estatistica/pesquisa.aspx")

Selecionar área de ocorrência para coleta

In [ ]:
# Seleciona Ocorrência Registrada por Mês
navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[5]/div[1]/div/a').send_keys(Keys.ENTER)

Criando tabela de dados

In [ ]:
# Transformando arquivo .txt em .csv para construir tabela principal
frame_main = pd.read_csv("municipios.txt", delimiter='\t')

# Adicionando coluna para inserção de dados
frame_main["Furto e Roubo"] = 0

Colhendo os dados

In [ ]:
# Laço para repetir município por município
for c1 in range(0,645): # c1 vai ser usado como índice para associar os municípios
    limpar() # Usado para manter somente o print da função abaixo
    carregamento(c1) # Usado para dizer para o usuário quanto que falta para finalizar
    coletor = [] # Variável para manipulação e armazenagem dos dados filtrados

    select_box1 = Select(navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[3]/div[1]/div/select')) # Identificação da Caixa de Seleção Suspensa do município
    select_box1.select_by_index(c1 + 1) # Selecionar o município pelo índice

    table = navegador.find_element(By.CSS_SELECTOR, '[id="conteudo_repAnos_gridDados_2"]') # Identificação da Tabela Principal 2023
    
    # Seleção, coleta, filtragem e manipulação dos dados
    trs = table.find_elements(By.TAG_NAME, "tr") # Selecionar as linhas da tabela
    for td in trs: # Percorrer cada linha da tabela

        # Condicional para capturar somente os números necessários
        if "TOTAL DE ROUBO - OUTROS (1)" in td.text:
            coletor.append(lastnum_sdot(td.text))
        if "ROUBO DE VEÍCULO" in td.text:
            coletor.append(lastnum_sdot(td.text))
        if "FURTO - OUTROS"	in td.text:
            coletor.append(lastnum_sdot(td.text))
        if "FURTO DE VEÍCULO"	in td.text:
            coletor.append(lastnum_sdot(td.text))
        
    input_frame = sum(coletor) # Somar todos os dados coletados de um município
    frame_main.loc[c1, 'Furto e Roubo'] = input_frame # Armazenar na tabela principal, sendo c1 o índice na coluna "Furto e Roubo"

    # Resetar para o estado limpo da página
    select_box2 = Select(navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[2]/div[2]/div/select')) # Identificação da Caixa de Seleção Suspensa de Região
    select_box2.select_by_index(0) # Selecionar "Todos" na caixa

Exportação da Tabela Principal para finalização do WebScraping

In [ ]:
# Produto final
frame_main.to_excel('produto_pronto2021.xlsx', index=False)